In [1]:
import pandas as pd
import numpy as np

In [23]:
def iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution, max_iterations=100, tol=1e-6, max_saturation=0.85):
    """
    Aplica IPF para distribuir el forecast anual a nivel mensual por tienda,
    incluyendo la distribución por tipo y categoría de pedido,
    respetando la capacidad máxima mensual por tienda y aplicando la restricción de saturación.
    Devuelve la matriz ajustada y el volumen de pedidos no asignados.
    """
    
    # Expandimos dimensiones para la multiplicación correcta
    historical_distribution_exp = historical_distribution[:, np.newaxis, np.newaxis, :]  # (3,1,1,12)
    type_distribution_exp = type_distribution[:, :, np.newaxis, np.newaxis]  # (3,2,1,1)
    category_distribution_exp = category_distribution[:, np.newaxis, :, np.newaxis]  # (3,1,3,1)
    
    # Inicializamos la matriz de pedidos basada en las distribuciones históricas
    forecast_monthly = forecast[:, np.newaxis, np.newaxis, np.newaxis] * historical_distribution_exp * type_distribution_exp * category_distribution_exp
    
    total_excess = 0  # Contador de pedidos no asignados
    
    for _ in range(max_iterations):
        # Ajuste por filas: restringimos a la capacidad máxima mensual por tienda
        row_totals = forecast_monthly.sum(axis=(1, 2), keepdims=True)  # Suma total por tienda y mes
        row_factors = np.minimum(capacity_max_monthly / row_totals, 1)  # Factor de ajuste para respetar capacidad
        forecast_monthly *= row_factors  # Aplicamos la restricción de capacidad
        
        # Ajuste por columnas: aseguramos que la distribución respete el forecast anual
        col_totals = forecast_monthly.sum(axis=(1, 2))  # Suma total por tienda
        col_factors = forecast / col_totals  # Factor de ajuste por tienda
        forecast_monthly *= col_factors[:, np.newaxis, np.newaxis, np.newaxis]
        
        # Aplicamos la restricción de saturación
        saturation = forecast_monthly.sum(axis=(1, 2)) / capacity_max_monthly
        excess_mask = saturation > max_saturation
        excess = np.maximum(forecast_monthly.sum(axis=(1, 2)) - (max_saturation * capacity_max_monthly), 0) * excess_mask
        total_excess += np.sum(excess)  # Acumulamos el exceso no asignado
        
        if np.sum(excess) == 0:
            break  # Si no hay excesos, terminamos
        
        # Identificar meses con espacio disponible
        available_space = (max_saturation * capacity_max_monthly) - forecast_monthly.sum(axis=(1, 2))
        available_space[available_space < 0] = 0
        
        # Redistribuir el exceso proporcionalmente al espacio disponible
        total_available_space = available_space.sum(axis=1, keepdims=True)
        redistribution_factors = np.divide(available_space, total_available_space, where=total_available_space > 0)
        forecast_monthly -= excess[:, np.newaxis, np.newaxis, :]  # Restar el exceso
        forecast_monthly += redistribution_factors[:, np.newaxis, np.newaxis, :] * excess[:, np.newaxis, :]  # Reasignar
    
    return forecast_monthly, total_excess

In [ ]:
# Datos de ejemplo
forecast = np.array([120000, 90000, 60000])  # Forecast anual por tienda
historical_distribution = np.array([
    [0.10, 0.08, 0.07, 0.09, 0.10, 0.07, 0.06, 0.08, 0.07, 0.09, 0.10, 0.09],  # Tienda A
    [0.09, 0.07, 0.08, 0.10, 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.10, 0.09],  # Tienda B
    [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.10, 0.10, 0.09]   # Tienda C
])

capacity_max_monthly = np.array([
    [12000, 11000, 10000, 13000, 14000, 12000, 11000, 12500, 11500, 13500, 14500, 13000],  # Tienda A
    [10000, 9500, 9000, 10500, 11000, 10000, 9500, 10200, 9900, 10800, 11200, 10500],  # Tienda B
    [8000, 8500, 7800, 8600, 9200, 8800, 8100, 8700, 8900, 9400, 9600, 9200]   # Tienda C
])

# Proporción histórica de pedidos por tipo y categoría
type_distribution = np.array([
    [0.6, 0.4],  # Tienda A
    [0.5, 0.5],  # Tienda B
    [0.7, 0.3]   # Tienda C
])

category_distribution = np.array([
    [0.5, 0.3, 0.2],  # Tienda A
    [0.4, 0.4, 0.2],  # Tienda B
    [0.6, 0.3, 0.1]   # Tienda C
])

In [11]:
# Datos de ejemplo
forecast = np.array([120000, 90000, 60000])  # Forecast anual por tienda
historical_distribution = np.array([
    [0.10, 0.08, 0.07, 0.09, 0.10, 0.07, 0.06, 0.08, 0.07, 0.09, 0.10, 0.09],  # Tienda A
    [0.09, 0.07, 0.08, 0.10, 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.10, 0.09],  # Tienda B
    [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.10, 0.10, 0.09]   # Tienda C
])

In [12]:
historical_distribution

array([[0.1 , 0.08, 0.07, 0.09, 0.1 , 0.07, 0.06, 0.08, 0.07, 0.09, 0.1 ,
        0.09],
       [0.09, 0.07, 0.08, 0.1 , 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.1 ,
        0.09],
       [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.1 , 0.1 ,
        0.09]])

In [13]:
capacity_max_monthly = np.array([
    [12000, 11000, 10000, 13000, 14000, 12000, 11000, 12500, 11500, 13500, 14500, 13000],  # Tienda A
    [10000, 9500, 9000, 10500, 11000, 10000, 9500, 10200, 9900, 10800, 11200, 10500],  # Tienda B
    [8000, 8500, 7800, 8600, 9200, 8800, 8100, 8700, 8900, 9400, 9600, 9200]   # Tienda C
])


In [14]:
# Proporción histórica de pedidos por tipo y categoría
# Suponiendo 2 tipos de pedidos y 3 categorías
# Dimensiones: [Tiendas, Tipos de pedidos, Categorías]
type_distribution = np.array([
    [0.6, 0.4],  # Tienda A
    [0.5, 0.5],  # Tienda B
    [0.7, 0.3]   # Tienda C
])

In [15]:

category_distribution = np.array([
    [0.5, 0.3, 0.2],  # Tienda A
    [0.4, 0.4, 0.2],  # Tienda B
    [0.6, 0.3, 0.1]   # Tienda C
])

In [24]:
# Ejecutar el algoritmo
forecast_adjusted, total_excess_orders = iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution)

# Convertir a DataFrame para visualización
months = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
shops = ["Tienda A", "Tienda B", "Tienda C"]
forecast_df = pd.DataFrame(forecast_adjusted.sum(axis=(1, 2, 3)), index=shops, columns=months)
print(forecast_df)
print(f"Total de pedidos no asignados: {total_excess_orders}")

ValueError: operands could not be broadcast together with shapes (3,) (3,12) 

In [22]:
# Ejecutar el algoritmo
forecast_adjusted, total_excess_orders = iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution)

# Convertir a DataFrame para visualización
months = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
shops = ["Tienda A", "Tienda B", "Tienda C"]
forecast_df = pd.DataFrame(forecast_adjusted.sum(axis=(1, 2, 3)), index=shops, columns=months)
print(forecast_df)
print(f"Total de pedidos no asignados: {total_excess_orders}")



ValueError: operands could not be broadcast together with shapes (3,) (3,12) 

In [ ]:
import numpy as np
import pandas as pd

def iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution, max_iterations=100, tol=1e-6, max_saturation=0.85):
    """
    Aplica IPF para distribuir el forecast anual a nivel mensual por tienda,
    incluyendo la distribución por tipo y categoría de pedido,
    respetando la capacidad máxima mensual por tienda y aplicando la restricción de saturación.
    Devuelve la matriz ajustada y el volumen de pedidos no asignados.
    """
    
    # Expandimos dimensiones para la multiplicación correcta
    historical_distribution_exp = historical_distribution[:, np.newaxis, np.newaxis, :]  # (3,1,1,12)
    type_distribution_exp = type_distribution[:, :, np.newaxis, np.newaxis]  # (3,2,1,1)
    category_distribution_exp = category_distribution[:, np.newaxis, :, np.newaxis]  # (3,1,3,1)
    
    # Inicializamos la matriz de pedidos basada en las distribuciones históricas
    forecast_monthly = forecast[:, np.newaxis, np.newaxis, np.newaxis] * historical_distribution_exp * type_distribution_exp * category_distribution_exp
    
    total_excess = 0  # Contador de pedidos no asignados
    
    for _ in range(max_iterations):
        # Ajuste por filas: restringimos a la capacidad máxima mensual por tienda
        row_totals = forecast_monthly.sum(axis=(1, 2), keepdims=True)  # Suma total por tienda y mes
        row_factors = np.minimum(capacity_max_monthly / row_totals, 1)  # Factor de ajuste para respetar capacidad
        forecast_monthly *= row_factors  # Aplicamos la restricción de capacidad
        
        # Ajuste por columnas: aseguramos que la distribución respete el forecast anual
        col_totals = forecast_monthly.sum(axis=(1, 2))  # Suma total por tienda
        col_factors = forecast / col_totals  # Factor de ajuste por tienda
        forecast_monthly *= col_factors[:, np.newaxis, np.newaxis, np.newaxis]
        
        # Aplicamos la restricción de saturación
        saturation = forecast_monthly.sum(axis=(1, 2)) / capacity_max_monthly
        excess_mask = saturation > max_saturation
        excess = np.maximum(forecast_monthly.sum(axis=(1, 2)) - (max_saturation * capacity_max_monthly), 0) * excess_mask
        total_excess += np.sum(excess)  # Acumulamos el exceso no asignado
        
        if np.sum(excess) == 0:
            break  # Si no hay excesos, terminamos
        
        # Identificar meses con espacio disponible
        available_space = (max_saturation * capacity_max_monthly) - forecast_monthly.sum(axis=(1, 2))
        available_space[available_space < 0] = 0
        
        # Redistribuir el exceso proporcionalmente al espacio disponible
        total_available_space = available_space.sum(axis=1, keepdims=True)
        redistribution_factors = np.divide(available_space, total_available_space, where=total_available_space > 0)
        forecast_monthly -= excess[:, np.newaxis, np.newaxis, :]  # Restar el exceso
        forecast_monthly += redistribution_factors[:, np.newaxis, np.newaxis, :] * excess[:, np.newaxis, :]  # Reasignar
    
    return forecast_monthly, total_excess

# Datos de ejemplo
forecast = np.array([120000, 90000, 60000])  # Forecast anual por tienda
historical_distribution = np.array([
    [0.10, 0.08, 0.07, 0.09, 0.10, 0.07, 0.06, 0.08, 0.07, 0.09, 0.10, 0.09],  # Tienda A
    [0.09, 0.07, 0.08, 0.10, 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.10, 0.09],  # Tienda B
    [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.10, 0.10, 0.09]   # Tienda C
])

capacity_max_monthly = np.array([
    [12000, 11000, 10000, 13000, 14000, 12000, 11000, 12500, 11500, 13500, 14500, 13000],  # Tienda A
    [10000, 9500, 9000, 10500, 11000, 10000, 9500, 10200, 9900, 10800, 11200, 10500],  # Tienda B
    [8000, 8500, 7800, 8600, 9200, 8800, 8100, 8700, 8900, 9400, 9600, 9200]   # Tienda C
])

# Proporción histórica de pedidos por tipo y categoría
type_distribution = np.array([
    [0.6, 0.4],  # Tienda A
    [0.5, 0.5],  # Tienda B
    [0.7, 0.3]   # Tienda C
])

category_distribution = np.array([
    [0.5, 0.3, 0.2],  # Tienda A
    [0.4, 0.4, 0.2],  # Tienda B
    [0.6, 0.3, 0.1]   # Tienda C
])

# Ejecutar el algoritmo
forecast_adjusted, total_excess_orders = iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution)

# Convertir a DataFrame para visualización
months = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
shops = ["Tienda A", "Tienda B", "Tienda C"]
forecast_df = pd.DataFrame(forecast_adjusted.sum(axis=(1, 2, 3)), index=shops, columns=months)
print(forecast_df)
print(f"Total de pedidos no asignados: {total_excess_orders}")


In [25]:
import numpy as np
import pandas as pd

def iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution, max_iterations=100, tol=1e-6, max_saturation=0.85):
    """
    Aplica IPF para distribuir el forecast anual a nivel mensual por tienda,
    incluyendo la distribución por tipo y categoría de pedido,
    respetando la capacidad máxima mensual por tienda y aplicando la restricción de saturación.
    Devuelve la matriz ajustada y el volumen de pedidos no asignados.
    """
    
    # Expandimos dimensiones para la multiplicación correcta
    historical_distribution_exp = historical_distribution[:, np.newaxis, np.newaxis, :]  # (3,1,1,12)
    type_distribution_exp = type_distribution[:, :, np.newaxis, np.newaxis]  # (3,2,1,1)
    category_distribution_exp = category_distribution[:, np.newaxis, :, np.newaxis]  # (3,1,3,1)
    
    # Inicializamos la matriz de pedidos basada en las distribuciones históricas
    forecast_monthly = forecast[:, np.newaxis, np.newaxis, np.newaxis] * historical_distribution_exp * type_distribution_exp * category_distribution_exp
    
    total_excess = 0  # Contador de pedidos no asignados
    
    for _ in range(max_iterations):
        # Ajuste por filas: restringimos a la capacidad máxima mensual por tienda
        row_totals = forecast_monthly.sum(axis=(1, 2), keepdims=True)  # Suma total por tienda y mes
        row_factors = np.minimum(capacity_max_monthly / row_totals, 1)  # Factor de ajuste para respetar capacidad
        forecast_monthly *= row_factors  # Aplicamos la restricción de capacidad
        
        # Ajuste por columnas: aseguramos que la distribución respete el forecast anual
        col_totals = forecast_monthly.sum(axis=(1, 2))  # Suma total por tienda
        col_factors = forecast / col_totals  # Factor de ajuste por tienda
        forecast_monthly *= col_factors[:, np.newaxis, np.newaxis, np.newaxis]
        
        # Aplicamos la restricción de saturación
        saturation = forecast_monthly.sum(axis=(1, 2)) / capacity_max_monthly
        excess_mask = saturation > max_saturation
        excess = np.maximum(forecast_monthly.sum(axis=(1, 2)) - (max_saturation * capacity_max_monthly), 0) * excess_mask
        total_excess += np.sum(excess)  # Acumulamos el exceso no asignado
        
        if np.sum(excess) == 0:
            break  # Si no hay excesos, terminamos
        
        # Identificar meses con espacio disponible
        available_space = (max_saturation * capacity_max_monthly) - forecast_monthly.sum(axis=(1, 2))
        available_space[available_space < 0] = 0
        
        # Redistribuir el exceso proporcionalmente al espacio disponible
        total_available_space = available_space.sum(axis=1, keepdims=True)
        redistribution_factors = np.divide(available_space, total_available_space, where=total_available_space > 0)
        forecast_monthly -= excess[:, np.newaxis, np.newaxis, :]  # Restar el exceso
        forecast_monthly += redistribution_factors[:, np.newaxis, np.newaxis, :] * excess[:, np.newaxis, :]  # Reasignar
    
    return forecast_monthly, total_excess

# Datos de ejemplo
forecast = np.array([120000, 90000, 60000])  # Forecast anual por tienda
historical_distribution = np.array([
    [0.10, 0.08, 0.07, 0.09, 0.10, 0.07, 0.06, 0.08, 0.07, 0.09, 0.10, 0.09],  # Tienda A
    [0.09, 0.07, 0.08, 0.10, 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.10, 0.09],  # Tienda B
    [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.10, 0.10, 0.09]   # Tienda C
])

capacity_max_monthly = np.array([
    [12000, 11000, 10000, 13000, 14000, 12000, 11000, 12500, 11500, 13500, 14500, 13000],  # Tienda A
    [10000, 9500, 9000, 10500, 11000, 10000, 9500, 10200, 9900, 10800, 11200, 10500],  # Tienda B
    [8000, 8500, 7800, 8600, 9200, 8800, 8100, 8700, 8900, 9400, 9600, 9200]   # Tienda C
])

# Proporción histórica de pedidos por tipo y categoría
type_distribution = np.array([
    [0.6, 0.4],  # Tienda A
    [0.5, 0.5],  # Tienda B
    [0.7, 0.3]   # Tienda C
])

category_distribution = np.array([
    [0.5, 0.3, 0.2],  # Tienda A
    [0.4, 0.4, 0.2],  # Tienda B
    [0.6, 0.3, 0.1]   # Tienda C
])

# Ejecutar el algoritmo
forecast_adjusted, total_excess_orders = iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution)

# Convertir a DataFrame para visualización
months = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
shops = ["Tienda A", "Tienda B", "Tienda C"]
forecast_df = pd.DataFrame(forecast_adjusted.sum(axis=(1, 2, 3)), index=shops, columns=months)
print(forecast_df)
print(f"Total de pedidos no asignados: {total_excess_orders}")


ValueError: operands could not be broadcast together with shapes (3,) (3,12) 

In [26]:
import numpy as np
import pandas as pd

def iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution, max_iterations=100, tol=1e-6, max_saturation=0.85):
    """
    Aplica IPF para distribuir el forecast anual a nivel mensual por tienda,
    incluyendo la distribución por tipo y categoría de pedido,
    respetando la capacidad máxima mensual por tienda y aplicando la restricción de saturación.
    Devuelve la matriz ajustada y el volumen de pedidos no asignados.
    """
    
    # Expandimos dimensiones para la multiplicación correcta
    historical_distribution_exp = historical_distribution[:, np.newaxis, np.newaxis, :]  # (3,1,1,12)
    type_distribution_exp = type_distribution[:, :, np.newaxis, np.newaxis]  # (3,2,1,1)
    category_distribution_exp = category_distribution[:, np.newaxis, :, np.newaxis]  # (3,1,3,1)
    
    # Inicializamos la matriz de pedidos basada en las distribuciones históricas
    forecast_monthly = forecast[:, np.newaxis, np.newaxis, np.newaxis] * historical_distribution_exp * type_distribution_exp * category_distribution_exp
    
    total_excess = 0  # Contador de pedidos no asignados
    
    for _ in range(max_iterations):
        # Ajuste por filas: restringimos a la capacidad máxima mensual por tienda
        row_totals = forecast_monthly.sum(axis=(1, 2), keepdims=True)  # Suma total por tienda y mes
        row_factors = np.minimum(capacity_max_monthly / row_totals, 1)  # Factor de ajuste para respetar capacidad
        forecast_monthly *= row_factors  # Aplicamos la restricción de capacidad
        
        # Ajuste por columnas: aseguramos que la distribución respete el forecast anual
        col_totals = forecast_monthly.sum(axis=(1, 2))  # Suma total por tienda
        col_factors = forecast / col_totals  # Factor de ajuste por tienda
        forecast_monthly *= col_factors[:, np.newaxis, np.newaxis, np.newaxis]
        
        # Aplicamos la restricción de saturación
        saturation = forecast_monthly.sum(axis=(1, 2)) / capacity_max_monthly
        excess_mask = saturation > max_saturation
        excess = np.maximum(forecast_monthly.sum(axis=(1, 2)) - (max_saturation * capacity_max_monthly), 0) * excess_mask
        total_excess += np.sum(excess)  # Acumulamos el exceso no asignado
        
        if np.sum(excess) == 0:
            break  # Si no hay excesos, terminamos
        
        # Identificar meses con espacio disponible
        available_space = (max_saturation * capacity_max_monthly) - forecast_monthly.sum(axis=(1, 2))
        available_space[available_space < 0] = 0
        
        # Redistribuir el exceso proporcionalmente al espacio disponible
        total_available_space = available_space.sum(axis=1, keepdims=True)
        redistribution_factors = np.divide(available_space, total_available_space, where=total_available_space > 0)
        forecast_monthly -= excess[:, np.newaxis, np.newaxis, :]  # Restar el exceso
        forecast_monthly += redistribution_factors[:, np.newaxis, np.newaxis, :] * excess[:, np.newaxis, :]  # Reasignar
    
    return forecast_monthly, total_excess

# Datos de ejemplo
forecast = np.array([120000, 90000, 60000])  # Forecast anual por tienda
historical_distribution = np.array([
    [0.10, 0.08, 0.07, 0.09, 0.10, 0.07, 0.06, 0.08, 0.07, 0.09, 0.10, 0.09],  # Tienda A
    [0.09, 0.07, 0.08, 0.10, 0.09, 0.06, 0.05, 0.07, 0.09, 0.11, 0.10, 0.09],  # Tienda B
    [0.08, 0.09, 0.07, 0.08, 0.09, 0.07, 0.06, 0.08, 0.09, 0.10, 0.10, 0.09]   # Tienda C
])

capacity_max_monthly = np.array([
    [12000, 11000, 10000, 13000, 14000, 12000, 11000, 12500, 11500, 13500, 14500, 13000],  # Tienda A
    [10000, 9500, 9000, 10500, 11000, 10000, 9500, 10200, 9900, 10800, 11200, 10500],  # Tienda B
    [8000, 8500, 7800, 8600, 9200, 8800, 8100, 8700, 8900, 9400, 9600, 9200]   # Tienda C
])

# Proporción histórica de pedidos por tipo y categoría
type_distribution = np.array([
    [0.6, 0.4],  # Tienda A
    [0.5, 0.5],  # Tienda B
    [0.7, 0.3]   # Tienda C
])

category_distribution = np.array([
    [0.5, 0.3, 0.2],  # Tienda A
    [0.4, 0.4, 0.2],  # Tienda B
    [0.6, 0.3, 0.1]   # Tienda C
])

# Ejecutar el algoritmo
forecast_adjusted, total_excess_orders = iterative_proportional_fitting(forecast, historical_distribution, capacity_max_monthly, type_distribution, category_distribution)

# Convertir a DataFrame para visualización
months = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
shops = ["Tienda A", "Tienda B", "Tienda C"]
forecast_df = pd.DataFrame(forecast_adjusted.sum(axis=(1, 2, 3)), index=shops, columns=months)
print(forecast_df)
print(f"Total de pedidos no asignados: {total_excess_orders}")


ValueError: operands could not be broadcast together with shapes (3,) (3,12) 

ValueError: the 'keepdims' parameter is not supported in the pandas implementation of sum()

KeyError: 0

In [31]:
import numpy as np
import pandas as pd

# 1. Forecast Anual de pedidos por año (2025-2029)
forecast_anual = {
    2025: 12000,
    2026: 13500,
    2027: 15000,
    2028: 16500,
    2029: 18000
}

# 2. Distribución histórica de pedidos por tienda
distribucion_tienda = pd.Series([0.30, 0.25, 0.20, 0.15, 0.10], index=[f'Tienda {i+1}' for i in range(5)])  # Tiendas 1 a 5

distribucion_tienda /= distribucion_tienda.sum()

# 3. Distribución mensual histórica por tienda
distribucion_mensual = pd.DataFrame([
    [0.08, 0.07, 0.06, 0.08, 0.09, 0.10, 0.09, 0.07, 0.06, 0.08, 0.07, 0.05],
    [0.07, 0.06, 0.05, 0.07, 0.08, 0.09, 0.08, 0.06, 0.05, 0.07, 0.06, 0.04],
    [0.10, 0.09, 0.08, 0.10, 0.11, 0.12, 0.11, 0.09, 0.08, 0.10, 0.09, 0.07],
    [0.05, 0.04, 0.03, 0.05, 0.06, 0.07, 0.06, 0.04, 0.03, 0.05, 0.04, 0.03],
    [0.04, 0.03, 0.02, 0.04, 0.05, 0.06, 0.05, 0.03, 0.02, 0.04, 0.03, 0.02]
], index=[f'Tienda {i+1}' for i in range(5)], columns=[f'Mes {i+1}' for i in range(12)])

distribucion_mensual = distribucion_mensual.div(distribucion_mensual.sum(axis=1), axis=0)

# 4. Capacidad máxima mensual de cada tienda (valores ficticios)
capacidad_maxima = np.random.randint(100, 300, size=(5, 12))

# 5. Distribución conocida de tipos y categorías de pedidos
distribucion_tipo_pedido = pd.Series([0.30, 0.70], index=["Tipo 1", "Tipo 2"])   # Ejemplo: 30% tipo 1, 70% tipo 2
distribucion_categoria_pedido = pd.Series([0.15, 0.20, 0.10, 0.25, 0.20, 0.10], index=["Cat 1", "Cat 2", "Cat 3", "Cat 4", "Cat 5", "Cat 6"])   # Ejemplo de 6 categorías

distribucion_tipo_pedido /= distribucion_tipo_pedido.sum()
distribucion_categoria_pedido /= distribucion_categoria_pedido.sum()

# 6. Función para aplicar el IPF con redistribución considerando restricciones de capacidad
def ajustar_pedidos(forecast_anual, distribucion_tienda, distribucion_mensual, capacidad, distribucion_tipo, distribucion_categoria):
    pedidos_resultantes = {}
    pedidos_sobrantes_anual = {}
    
    for year, total_pedidos in forecast_anual.items():
        pedidos_por_tienda = total_pedidos * distribucion_tienda  # Distribuir primero por tienda
        matriz_pedidos = np.zeros((len(distribucion_tienda), 12))
        
        for i in range(len(distribucion_tienda)):  # Iterar sobre tiendas
            matriz_pedidos[i, :] = pedidos_por_tienda.iloc[i] * distribucion_mensual.iloc[i].values
        
        pedidos_sobrantes = np.zeros_like(matriz_pedidos)
        
        for i in range(matriz_pedidos.shape[0]):  # Iterar sobre tiendas
            for j in range(matriz_pedidos.shape[1]):  # Iterar sobre meses
                if matriz_pedidos[i, j] > capacidad[i, j] * 0.85:  # Restricción del 85%
                    exceso = matriz_pedidos[i, j] - (capacidad[i, j] * 0.85)
                    matriz_pedidos[i, j] = capacidad[i, j] * 0.85
                    pedidos_sobrantes[i, j] += exceso
        
        pedidos_por_tipo_categoria = {}
        for tipo_idx, tipo in enumerate(["Tipo 1", "Tipo 2"]):
            for cat_idx, categoria in enumerate(["Cat 1", "Cat 2", "Cat 3", "Cat 4", "Cat 5", "Cat 6"]):
                pedidos_por_tipo_categoria[(tipo, categoria)] = matriz_pedidos * distribucion_tipo[tipo_idx] * distribucion_categoria[cat_idx]
        
        pedidos_resultantes[year] = pedidos_por_tipo_categoria
        pedidos_sobrantes_anual[year] = pedidos_sobrantes.sum()
    
    return pedidos_resultantes, pedidos_sobrantes_anual

# 7. Aplicar el modelo
resultados, sobrantes = ajustar_pedidos(
    forecast_anual, distribucion_tienda, distribucion_mensual, capacidad_maxima,
    distribucion_tipo_pedido, distribucion_categoria_pedido
)

# 8. Mostrar resultados
for year in resultados:
    print(f'Pedidos ajustados para {year}:')
    for tipo, categoria in resultados[year]:
        print(f'Tipo: {tipo}, Categoría: {categoria}')
        print(pd.DataFrame(resultados[year][(tipo, categoria)], columns=[f'Mes {i+1}' for i in range(12)]))
    print(f'Pedidos sobrantes en {year}: {sobrantes[year]}')
    print('\n' + '-'*50 + '\n')


Pedidos ajustados para 2025:
Tipo: Tipo 1, Categoría: Cat 1
      Mes 1     Mes 2     Mes 3     Mes 4    Mes 5      Mes 6    Mes 7  \
0  7.611750  7.305750  8.797500  6.158250  6.61725   8.223750  7.22925   
1  7.114500  7.191000  8.653846  9.906750  6.08175   9.180000  7.07625   
2  9.473684  7.573500  7.578947  9.473684  9.67725  10.786500  6.69375   
3  7.038000  5.890909  4.418182  4.513500  8.33850   7.152750  3.90150   
4  5.023256  3.767442  2.511628  5.023256  4.59000   7.534884  6.27907   

      Mes 8     Mes 9    Mes 10    Mes 11    Mes 12  
0  7.114500  6.808500  9.103500  9.868500  8.912250  
1  8.874000  8.653846  6.923250  5.431500  6.923077  
2  8.526316  7.578947  9.473684  8.526316  6.540750  
3  3.978000  4.322250  7.363636  5.890909  4.418182  
4  3.767442  2.511628  5.023256  3.767442  2.511628  
Tipo: Tipo 1, Categoría: Cat 2
       Mes 1      Mes 2      Mes 3      Mes 4   Mes 5      Mes 6     Mes 7  \
0  10.149000   9.741000  11.730000   8.211000   8.823  10.9650

In [32]:
import numpy as np
import pandas as pd

# 1. Forecast Anual de pedidos por año (2025-2029)
forecast_anual = {
    2025: 12000,
    2026: 13500,
    2027: 15000,
    2028: 16500,
    2029: 18000
}

# 2. Distribución histórica de pedidos por tienda
distribucion_tienda = pd.Series([0.30, 0.25, 0.20, 0.15, 0.10], index=[f'Tienda {i+1}' for i in range(5)])  # Tiendas 1 a 5

distribucion_tienda /= distribucion_tienda.sum()

# 3. Distribución mensual histórica por tienda
distribucion_mensual = pd.DataFrame([
    [0.08, 0.07, 0.06, 0.08, 0.09, 0.10, 0.09, 0.07, 0.06, 0.08, 0.07, 0.05],
    [0.07, 0.06, 0.05, 0.07, 0.08, 0.09, 0.08, 0.06, 0.05, 0.07, 0.06, 0.04],
    [0.10, 0.09, 0.08, 0.10, 0.11, 0.12, 0.11, 0.09, 0.08, 0.10, 0.09, 0.07],
    [0.05, 0.04, 0.03, 0.05, 0.06, 0.07, 0.06, 0.04, 0.03, 0.05, 0.04, 0.03],
    [0.04, 0.03, 0.02, 0.04, 0.05, 0.06, 0.05, 0.03, 0.02, 0.04, 0.03, 0.02]
], index=[f'Tienda {i+1}' for i in range(5)], columns=[f'Mes {i+1}' for i in range(12)])

distribucion_mensual = distribucion_mensual.div(distribucion_mensual.sum(axis=1), axis=0)

# 4. Capacidad máxima mensual de cada tienda (valores ficticios)
capacidad_maxima = np.random.randint(100, 300, size=(5, 12))

# 5. Distribución conocida de tipos y categorías de pedidos
distribucion_tipo_pedido = pd.Series([0.30, 0.70], index=["Tipo 1", "Tipo 2"])   # Ejemplo: 30% tipo 1, 70% tipo 2
distribucion_categoria_pedido = pd.Series([0.15, 0.20, 0.10, 0.25, 0.20, 0.10], index=["Cat 1", "Cat 2", "Cat 3", "Cat 4", "Cat 5", "Cat 6"])   # Ejemplo de 6 categorías

distribucion_tipo_pedido /= distribucion_tipo_pedido.sum()
distribucion_categoria_pedido /= distribucion_categoria_pedido.sum()

# 6. Función para aplicar el IPF con redistribución considerando restricciones de capacidad
def ajustar_pedidos(forecast_anual, distribucion_tienda, distribucion_mensual, capacidad, distribucion_tipo, distribucion_categoria):
    pedidos_resultantes = {}
    pedidos_sobrantes_anual = {}
    
    for year, total_pedidos in forecast_anual.items():
        pedidos_por_tienda = total_pedidos * distribucion_tienda  # Distribuir primero por tienda
        matriz_pedidos = np.zeros((len(distribucion_tienda), 12))
        
        for i in range(len(distribucion_tienda)):  # Iterar sobre tiendas
            matriz_pedidos[i, :] = pedidos_por_tienda.iloc[i] * distribucion_mensual.iloc[i].values
        
        pedidos_sobrantes = np.zeros_like(matriz_pedidos)
        
        for i in range(matriz_pedidos.shape[0]):  # Iterar sobre tiendas
            for j in range(matriz_pedidos.shape[1]):  # Iterar sobre meses
                if matriz_pedidos[i, j] > capacidad[i, j] * 0.85:  # Restricción del 85%
                    exceso = matriz_pedidos[i, j] - (capacidad[i, j] * 0.85)
                    matriz_pedidos[i, j] = capacidad[i, j] * 0.85
                    pedidos_sobrantes[i, j] += exceso
        
        pedidos_por_tipo_categoria = {}
        for tipo_idx, tipo in enumerate(["Tipo 1", "Tipo 2"]):
            for cat_idx, categoria in enumerate(["Cat 1", "Cat 2", "Cat 3", "Cat 4", "Cat 5", "Cat 6"]):
                pedidos_por_tipo_categoria[(tipo, categoria)] = matriz_pedidos * distribucion_tipo[tipo_idx] * distribucion_categoria[cat_idx]
        
        pedidos_resultantes[year] = pedidos_por_tipo_categoria
        pedidos_sobrantes_anual[year] = pedidos_sobrantes.sum()
    
    return pedidos_resultantes, pedidos_sobrantes_anual

# 7. Aplicar el modelo
resultados, sobrantes = ajustar_pedidos(
    forecast_anual, distribucion_tienda, distribucion_mensual, capacidad_maxima,
    distribucion_tipo_pedido, distribucion_categoria_pedido
)

# 8. Mostrar resultados
for year in resultados:
    print(f'Pedidos ajustados para {year}:')
    for tipo, categoria in resultados[year]:
        print(f'Tipo: {tipo}, Categoría: {categoria}')
        print(pd.DataFrame(resultados[year][(tipo, categoria)], columns=[f'Mes {i+1}' for i in range(12)]))
    print(f'Pedidos sobrantes en {year}: {sobrantes[year]}')
    print('\n' + '-'*50 + '\n')


Pedidos ajustados para 2025:
Tipo: Tipo 1, Categoría: Cat 1
      Mes 1      Mes 2     Mes 3      Mes 4    Mes 5     Mes 6    Mes 7  \
0  5.240250  11.016000  9.333000  10.671750  7.42050  4.398750  8.98875   
1  7.994250   8.606250  8.653846   6.196500  7.07625  8.606250  9.14175   
2  9.473684   8.526316  6.617250   5.622750  7.07625  8.376750  4.09275   
3  6.158250   5.890909  4.418182   7.363636  7.30575  4.245750  6.92325   
4  5.023256   3.767442  2.511628   4.781250  6.27907  7.534884  5.12550   

      Mes 8     Mes 9    Mes 10     Mes 11    Mes 12  
0  8.606250  5.622750  8.912250  10.098000  5.775750  
1  9.524250  7.191000  7.305750   5.852250  6.923077  
2  5.584500  5.125500  9.473684   8.526316  5.508000  
3  5.890909  4.418182  7.152750   5.890909  4.418182  
4  3.767442  2.511628  5.023256   3.767442  2.511628  
Tipo: Tipo 1, Categoría: Cat 2
       Mes 1      Mes 2      Mes 3      Mes 4     Mes 5      Mes 6   Mes 7  \
0   6.987000  14.688000  12.444000  14.229000  9.8